# Creation of the Dataset

### Importations 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import IPython
import pyroomacoustics as pra
import pickle as pkl
import pandas as pd
from scipy.io import wavfile
from scipy.signal import fftconvolve
from scipy import signal
from matplotlib import gridspec

### Function that computes the absorption out of the room dimensions and the RT60.

In [2]:
def compute_absorption(room_eadge,room_height,RT60):
    V = (room_eadge**2)*room_height
    S = 2*(room_eadge**2) + 4*(room_height**2)
    absr = 0.1611*(V/(S*RT60))
    return absr

### Setting the different parameters for the rooms

In [3]:
#Here You should put the path to where the file wav is on the machine you are working on.
folder_path = "/home/janjar/Semester_Project/Dataset/GDWN/WGN.wav"
file_to_open = folder_path 

ang = (np.array([61.]) / 180.) * np.pi
distance = 1.0
a = np.array([5.,5.,3.])
b = np.array([6.,6.,3.])
dimensions = np.stack((a,b))
distances = np.array([1.,2.])
RT60 = np.array([0.2,0.3])

absorption_1 = compute_absorption(a[0],a[2],RT60[0])
absorption_2 = compute_absorption(b[0],b[2],RT60[1])
absorptions = np.array([absorption_1,absorption_2])
print(absorptions.shape)

snr = 5.
fs,signal_wav = wavfile.read(file_to_open)
snrs = np.array([5,10,15,20])

(2,)


## Methode that generates a room out of the conditions given in argument.
### Returns the audio_signals for the 4 mics, the location of the source and the absorption of the room .

In [4]:
def generate_room_from_conditions(ang,distance,absorptions,snr,dimensions,typeofroom):
   
    room_dim = dimensions[typeofroom]
    room_eadge = room_dim[0]

    sf = (0.03*np.sqrt(2)/4)
    corners = np.array([[0,0],[0,room_eadge],[room_eadge,room_eadge],[room_eadge,0]]).T
    
    absr = absorptions[typeofroom]
    room = pra.Room.from_corners(corners, fs=fs , max_order=8 ,absorption=absorptions[typeofroom])
    room.extrude(3.)

    # Add sources of 1 second duration
    #room_dim = np.array([5,5,3])
    rng = np.random.RandomState(23)
    duration_samples = int(fs)


    source_location = room_dim / 2 + (distance * np.array([np.cos(ang), np.sin(ang),0.0]))
    source_signal = rng.randn(duration_samples)
    room.add_source(source_location, signal = signal_wav)
    print('Here is the source at this step:',source_location) 

    #We initiate the point of the Tethra then we scale them
    #and translate the origin of the Tethra to the center of the room.
    R = np.array([(sf*np.array([1,1,-1,-1]))+2.5,(sf*np.array([1,-1,1,-1]))+2.5,(sf*np.array([1,-1,-1,1]))+1.5])# [[x], [y], [z]]
    room.add_microphone_array(pra.MicrophoneArray(R,room.fs))
    room.image_source_model(use_libroom=True)

    #Visualization
    #fig, ax = room.plot(img_order=3)
    #fig.set_size_inches(18.5,10.5)

    room.simulate(snr = snr)
    #print('Here is the audio signal:',room.mic_array.signals[0][:5])
    return room.mic_array.signals,source_location,absr

### Testing that generate_room works properly

In [5]:
ang = (np.array([61.]) / 180.) * np.pi
audio_signals,source_loc,t = generate_room_from_conditions(ang,distances[0],absorptions,snr,dimensions,0)
print(audio_signals[0,:].shape)

Here is the source at this step: [array([2.98480962]) array([3.37461971]) 1.5]
(49702,)


## Method that compute the phasematrix out of a signal and store it in the training folder.
### Returns the paths to the audio_file and the phase_matrix.

In [6]:
def generate_phasematrix_from_signals(signals,j):
    
    #Here also the path should be the folder you need the audio files to be stored on.
        path = "/home/janjar/Dataset/Trainingset/"   
        name_signals = 'audio_signals/audio_signals-{}'.format(j)
        fileName_audio = path + name_signals
        fileObject = open(fileName_audio, 'wb')
        pkl.dump(signals, fileObject)
        fileObject.close()                
    
        phase_matrix = np.empty((4, 129, 390))
        for i in range(4):
            f, t, stft_mic0 = signal.stft(audio_signals[i,:].astype(np.float32), fs)
            spectrum = stft_mic0
            magnitude = np.abs(spectrum)
            phase = np.angle(spectrum)
            #print('Here is the phase:',phase)
            x = phase.shape[0]
            y = phase.shape[1]
            #print("HERE IS THE SHAPE",phase.shape)
            #phase_matrix = np.empty((4, x, y))
            phase_matrix[i] = phase
            
        path = "/home/janjar/Dataset/Trainingset/"   
        name_matrix = 'phase_matrix/Phase_matrix-{}'.format(j)
        fileName_matrix = path + name_matrix
        fileObject = open(fileName_matrix, 'wb')
        pkl.dump(phase_matrix, fileObject)
        fileObject.close()                    
        return fileName_audio,fileName_matrix

### Testing that generate_phasematrix works properly

In [7]:
path_to_matrix = generate_phasematrix_from_signals(audio_signals,0)
print(path_to_matrix)

('/home/janjar/Dataset/Trainingset/audio_signals/audio_signals-0', '/home/janjar/Dataset/Trainingset/phase_matrix/Phase_matrix-0')


### Funtion to assign the right label to each angle.

In [8]:
def assign_label_to_anlges(angles):
	bins = np.linspace(0, 180, 37)
	label = np.digitize(angles, bins)
	return label

## The main function that generate the entire Dataset set including the Dataframe to represent it.
### Return the Dataframe representing the training set.

In [9]:
def genere_dataset():
        index = 0
        df = pd.DataFrame(columns = ['Room','Array_position','Distance','Absorption','SNR','Audio_file','Phase_Matrix','Label'])
        snr = np.array([0,5,10,15,20]) #Comment this part to turn the snr into random assignment for each room
	#We iterate over the type of rooms (R1,R2).
        for i in range (2):          
	#We iterate over the distances from the microphones (1m,2m).	
            for j in range(2):
    #We iterate over the 5 different SNRS.       
                for l in range(5):    
	#We iterate on the 7 random positions of the source.		
                    for k in range(7):
                        print('Room:',index)
                        angles = np.random.uniform(low=0, high=180, size=(7,))
                        #print('Here are the angles for this Room:',angles) #Make the angles able to go to floats.
                        labels = assign_label_to_anlges(angles)
                        #print('Here are the labels for this Room:',labels) #Check if the labels match the angles.
                        azimuth = angles / 180. * np.pi 
                        #snr = snrs[np.random.randint(4,size = 1)] # Uncomment this part to turn the snr into random assignment for each room
                        signals,array_position,absr = generate_room_from_conditions(angles[k],distances[j],absorptions,snr[l],dimensions,i)
                        fileName_audio,fileName_matrix = generate_phasematrix_from_signals(signals,index)
                        df = df.append({'Room':i,'Array_position':array_position,'Distance':distances[j],'Absorption':absr,'SNR':snr[l],'Audio_file':fileName_audio,'Phase_Matrix':fileName_matrix,'Label':labels[k]} , ignore_index=True)
                        index = index+1
        return df               



In [10]:
Dataset = genere_dataset()

Room: 0
Here is the source at this step: [3.47179195 2.26416019 1.5       ]
Room: 1
Here is the source at this step: [1.51758729 2.31327755 1.5       ]
Room: 2
Here is the source at this step: [2.47633763 3.49972001 1.5       ]
Room: 3
Here is the source at this step: [2.95853825 3.38867467 1.5       ]
Room: 4
Here is the source at this step: [2.88336576 3.42359661 1.5       ]
Room: 5
Here is the source at this step: [3.15636155 1.7455535  1.5       ]
Room: 6
Here is the source at this step: [3.11505327 3.28848556 1.5       ]
Room: 7
Here is the source at this step: [1.89902143 1.70073487 1.5       ]
Room: 8
Here is the source at this step: [3.14945063 1.73959624 1.5       ]
Room: 9
Here is the source at this step: [2.92165939 1.5932457  1.5       ]
Room: 10
Here is the source at this step: [3.30693303 3.09064295 1.5       ]
Room: 11
Here is the source at this step: [2.29128515 1.52202346 1.5       ]
Room: 12
Here is the source at this step: [3.3578262 3.0139399 1.5      ]
Room: 13
Her

Room: 108
Here is the source at this step: [4.82215798 3.82446365 1.5       ]
Room: 109
Here is the source at this step: [3.94706053 4.7615551  1.5       ]
Room: 110
Here is the source at this step: [4.7611629  2.05221034 1.5       ]
Room: 111
Here is the source at this step: [2.51392305 1.0599667  1.5       ]
Room: 112
Here is the source at this step: [4.73466352 3.99546093 1.5       ]
Room: 113
Here is the source at this step: [4.99442272 3.14925819 1.5       ]
Room: 114
Here is the source at this step: [4.99371556 3.15842433 1.5       ]
Room: 115
Here is the source at this step: [3.13513828 1.00457081 1.5       ]
Room: 116
Here is the source at this step: [1.03074535 3.34933669 1.5       ]
Room: 117
Here is the source at this step: [1.10389901 3.63623975 1.5       ]
Room: 118
Here is the source at this step: [2.22611883 1.1557907  1.5       ]
Room: 119
Here is the source at this step: [1.15368267 2.23116172 1.5       ]
Room: 120
Here is the source at this step: [4.29263465 4.5261375

In [12]:
Dataset

,Room,Array_position,Distance,Absorption,SNR,Audio_file,Phase_Matrix,Label
0,0,"[3.4717919456626856, 2.264160193467828, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,9
1,0,"[1.5175872935403802, 2.313277547716712, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,18
2,0,"[2.476337628810577, 3.499720006896778, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,7
3,0,"[2.958538251046295, 3.388674671816073, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,28
4,0,"[2.8833657635145347, 3.423596606406129, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,22
5,0,"[3.1563615451236733, 1.745553499522422, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,3
6,0,"[3.1150532662594665, 3.28848556084659, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,4
7,0,"[1.8990214262966891, 1.70073486629934, 1.5]",1.0,0.702471,5,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,3
8,0,"[3.1494506348470934, 1.7395962435017123, 1.5]",1.0,0.702471,5,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,32
9,0,"[2.921659392138763, 1.5932457019560542, 1.5]",1.0,0.702471,5,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,34


### Saving the Dataframe into a pickle to use it for the Training

In [15]:
Dataset.to_pickle("/home/janjar/Dataset/Trainingset/Training_dataframe.pkl")